In [1]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
# nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
# ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

sys.path.append(os.path.abspath("../../ml/"))
from apply_loocv_and_save import load_and_preprocess
from dev_utils import read_yaml_for_host, filter_dev_data
import warnings


config_data = read_yaml_for_host("sst_config.yml")




python initialized for apply_loocv_and_save
cpus available; cpus to use:
28 25
28
yaml file does not contain host-specific settings for this host. Using default settings.


In [3]:
nonbids_data_path = config_data['nonbids_data_path']
ml_data_folderpath = nonbids_data_path + "fMRI/ml"
dropbox_datapath = config_data['dropbox_datapath']



In [4]:
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20230102T164214.csv")
train_subjs = filter_dev_data(test_train_df_raw, nonbids_data_path,dropbox_datapath)


In [5]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [6]:
from dev_wtp_io_utils import import_sst_betaseries_w1_subjs_to_pkl

In [7]:
all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")
#cut out other waves
sst_1_behavdesign_1 = all_sst_events[(all_sst_events.wave=='wave1')].reset_index(drop=True)
#label the ITI as such
sst_1_behavdesign_1.loc[(all_sst_events.trial_type.isna()),'trial_type']='ITI'
#ensure it is correctly ordered
sst_1_behavdesign_clean = sst_1_behavdesign_1.sort_values(['subject','onset']).reset_index(drop=True)
#now tell it the name of the corresponding beta. this is easy; they are all consecutive
sst_1_behavdesign_clean['beta'] = 'beta_' + (sst_1_behavdesign_clean.groupby(['subject','wave']).cumcount()+1).astype(str).apply(lambda x: x.zfill(4)) + '.nii'

In [8]:
#I want to import this other module; we don't actually use it but it's a good reference
#for future programmers to see
#it's important we keep the logic consistent for the two main conditions
sys.path.append(os.path.abspath("../../fx/multiconds/SST/"))
from multiconds import create_posterror_masks_from_masks

#sst_1_behavdesign_clean['prev_trial_type'] = sst_1_behavdesign_clean['trial_type']
sst_1_behavdesign_clean['prev_trial_type'] = (
    sst_1_behavdesign_clean
    .loc[sst_1_behavdesign_clean['trial_type']!='ITI']
    .groupby(['subject','wave'])
    .shift(1)
    ['trial_type']
)#this works because during assignment, items will be assigned to the line corresponding to their index
#very important you don't reset index during this operation.
is_post_stop_correct_go = (
    (sst_1_behavdesign_clean['trial_type']=='correct-go') & 
    (sst_1_behavdesign_clean['prev_trial_type'].isin(['failed-stop','correct-stop']))
)

sst_1_behavdesign_clean.loc[is_post_stop_correct_go,'post_stop_correct_go_subtype'] = 'Other'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='failed-stop'),'post_stop_correct_go_subtype'] = 'post-error'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='correct-stop'),'post_stop_correct_go_subtype'] = 'post-correct'


In [9]:
sst_1_behavdesign_clean.groupby(['trial_type','post_stop_correct_go_subtype']).size().reset_index()

,trial_type,post_stop_correct_go_subtype,0
0,correct-go,post-correct,1879
1,correct-go,post-error,2145


In [10]:
sst_1_behavdesign_clean['trial_type'].value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [11]:
behavdesign = sst_1_behavdesign_clean
sst_1_behavdesign_clean_dev005 = behavdesign[[s in 'DEV005' for s in behavdesign.subject]]

## Just subjects in the conditions of interest: CorrectStop vs. CorrectGo

In [12]:
sst_1_behavdesign_clean.trial_type.value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [13]:
sst_1_behavdesign_correct_conditions=sst_1_behavdesign_clean[sst_1_behavdesign_clean.trial_type.isin(['correct-go','correct-stop'])]

In [14]:
sst_1_behavdesign_correct_conditions.trial_type.value_counts()

correct-go      18750
correct-stop     3094
Name: trial_type, dtype: int64

In [15]:
# #we have data missing fro mthese subjects below. not enstirely sure yet; see 
# #https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq


# train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

### Now mask those subjects

In [16]:
# get the PFC mask
#mask_pfc = nib.load(ml_data_folderpath + '/prefrontal_cortex.nii.gz')

In [17]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:2],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':"beta"
    }
    )

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV020
...................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
4098    2.75834   0.75769    correct-go  DEV020  wave1  beta_0003.nii   
4100    5.51390   0.53414    correct-go  DEV020  wave1  beta_0005.nii   
4102    9.02224   0.60132    correct-go  DEV020  wave1  beta_0007.nii   
4104   12.52780   0.62845    correct-go  DEV020  wave1  beta_0009.nii   
4106   15.90975   0.63436    correct-go  DEV020  wave1  beta_0011.nii   
...         ...       ...           ...     ...    ...            ...   
4340  401.49167   2.00556  correct-stop  DEV020  

In [19]:
print("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:6],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':"beta"
    }
)

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV020
...................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
temp mask created.
loading files. This step may take some time...


KeyboardInterrupt: 

In [20]:
# print ("running")
# print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:30],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':"beta"
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV020
...................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
4098    2.75834   0.75769    correct-go  DEV020  wave1  beta_0003.nii   
4100    5.51390   0.53414    correct-go  DEV020  wave1  beta_0005.nii   
4102    9.02224   0.60132    correct-go  DEV020  wave1  beta_0007.nii   
4104   12.52780   0.62845    correct-go  DEV020  wave1  beta_0009.nii   
4106   15.90975   0.63436    correct-go  DEV020  wave1  beta_0011.nii   
...         ...       ...           ...     ...    ...            ...   
4340  401.49167   2.00556  correct-stop  DEV020  wave1  beta_024

...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
17920    0.00000   2.25834  correct-stop  DEV075  wave1  beta_0001.nii   
17922    2.75834   0.94359    correct-go  DEV075  wave1  beta_0003.nii   
17924    5.51390   0.59314    correct-go  DEV075  wave1  beta_0005.nii   
17926    9.02224   0.46077    correct-go  DEV075  wave1  beta_0007.nii   
17928   12.52780   0.53969    correct-go  DEV075  wave1  beta_0009.nii   
...          ...       ...           ...     ...    ...            ...   
18162  398.86111   0.51157    correct-go  DEV075  wave1  beta_0243.nii   
18166  403.99723   0.50196    correct-go  DEV075  wave1  beta_0247.nii   
18168  

............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0005.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
24324    6.38890   0.95380    correct-go  DEV102  wave1  beta_0005.nii   
24330   20.53475   0.52267    correct-go  DEV102  wave1  beta_0011.nii   
24332   23.35212   1.00374    correct-go  DEV102  wave1  beta_0013.nii   
24334   25.98407   0.59099    correct-go  DEV102  wave1  beta_0015.nii   
24336   28.92644   0.50155    correct-go  DEV102  wave1  beta_0017.nii   
...          ...       ...           ...     ...    ...            ...   
24562  400.61111   0.81409    correct-go  DEV102  wave1  beta_0243.nii   
24564  404.11667   0.77791    correct-go  DEV102  wave1  beta_0245.nii   
24568  409

...................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
27392    0.00000   0.88479    correct-go  DEV114  wave1  beta_0001.nii   
27396    6.38890   0.44063    correct-go  DEV114  wave1  beta_0005.nii   
27400   15.65280   0.59958    correct-go  DEV114  wave1  beta_0009.nii   
27402   20.53475   0.39786    correct-go  DEV114  wave1  beta_0011.nii   
27404   23.35212   0.39660    correct-go  DEV114  wave1  beta_0013.nii   
...          ...       ...           ...     ...    ...            ...   
27636  404.11667   0.52210    correct-go  DEV114  wave1  beta_0245.nii   
27638  407.24723   0.42238    correct-go  DEV114  wave1  beta_0247.nii   
27642  412.44516   

...................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
34050    2.75834   0.60594    correct-go  DEV141  wave1  beta_0003.nii   
34052    5.51390   0.40100    correct-go  DEV141  wave1  beta_0005.nii   
34054    9.02224   0.45223    correct-go  DEV141  wave1  beta_0007.nii   
34056   12.52780   0.44496    correct-go  DEV141  wave1  beta_0009.nii   
34058   15.90975   0.41975    correct-go  DEV141  wave1  beta_0011.nii   
...          ...       ...           ...     ...    ...            ...   
34294  403.99723   0.49383    correct-go  DEV141  wave1  beta_0247.nii   
34296  407.31460   0.48169    correct-go  DEV141  wave1  beta_0249.nii   
342

............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration  trial_type subject   wave           beta  \
37376    0.00000   0.87197  correct-go  DEV157  wave1  beta_0001.nii   
37380    6.38890   0.08329  correct-go  DEV157  wave1  beta_0005.nii   
37384   15.65280   0.59839  correct-go  DEV157  wave1  beta_0009.nii   
37386   20.53475   0.46574  correct-go  DEV157  wave1  beta_0011.nii   
37388   23.35212   0.34208  correct-go  DEV157  wave1  beta_0013.nii   
...          ...       ...         ...     ...    ...            ...   
37618  400.61111   0.43916  correct-go  DEV157  wave1  beta_0243.nii   
37620  404.11667   0.72024  correct-go  DEV157  wave1  beta_0245.nii   
37622  407.24723   0.43989  

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_betaseries_nos_30subs_correct_cond.pkl


In [ ]:
# print ("running")
# print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs,'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':"beta"
    }
)

### Standardize by condition

In [ ]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:2],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_condstd',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':"beta"
    }
    )

In [ ]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:6],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_condstd',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':"beta"
    }
    )

In [21]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:30],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_condstd',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':"beta"
    }
    )

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV020
...................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
0.57504374
127.23113
0.0029207596
1.0164568
          onset  duration    trial_type subject   wave           beta  \
4098    2.75834   0.75769    correct-go  DEV020  wave1  beta_0003.nii   
4100    5.51390   0.53414    correct-go  DEV020  wave1  beta_0005.nii   
4102    9.02224   0.60132    correct-go  DEV020  wave1  beta_0007.nii   
4104   12.52780   0.62845    correct-go  DEV020  wave1  beta_0009.nii   
4106   15.90975   0.63436    correct-go  DEV020  wave1  beta_0011.nii   
...         ...       ...           ...     ...    ...            ...   
4340 

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:576: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-2.5912359
124.141235
-0.0011770204
1.0066711
           onset  duration    trial_type subject   wave           beta  \
11008    0.00000   0.96259    correct-go  DEV047  wave1  beta_0001.nii   
11012    6.38890   0.58196    correct-go  DEV047  wave1  beta_0005.nii   
11016   15.65280   0.60373    correct-go  DEV047  wave1  beta_0009.nii   
11018   20.53475   0.56612    correct-go  DEV047  wave1  beta_0011.nii   
11020   23.35212   0.63492    correct-go  DEV047  wave1  beta_0013.nii   
...          ...       ...           ...     ...    ...            ...   
11252  404.11667   0.50827    correct-go  DEV047  wave1  beta_0245.nii   
11254  407.24723   0.55187    correct-go  DEV047  wave1  beta_0247.nii   
11256  409.68960   2.00556  correct-stop  DEV047  wave1  beta_0249.nii   
11258  412.44516   0.54573    correct-go  DEV047  w

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:576: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


retrieving a mask from the first image in the series beta_0003.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
1.2515695
95.559845
-0.0007336632
0.97317255
           onset  duration  trial_type subject   wave           beta  \
14850    2.75834   0.69411  correct-go  DEV063  wave1  beta_0003.nii   
14852    5.51390   0.55317  correct-go  DEV063  wave1  beta_0005.nii   
14854    9.02224   0.51317  correct-go  DEV063  wave1  beta_0007.nii   
14856   12.52780   0.45418  correct-go  DEV063  wave1  beta_0009.nii   
14858   15.90975   0.45353  correct-go  DEV063  wave1  beta_0011.nii   
...          ...       ...         ...     ...    ...            ...   
15090  398.86111   0.55657  correct-go  DEV063  wave1  beta_0243.nii   
15094  403.99723   0.53123  correct-go  DEV063  wave1  beta_0247.nii   
15096  407.31460   0.44475  correct-go  DEV063  wave1  beta_0249.nii   
15098  410.82016   0.51348  correct-go  DEV063  wave1  beta_0251.nii   


...

retrieving a mask from the first image in the series beta_0005.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-1.3591342
123.53089
-0.0023149492
1.0091861
           onset  duration    trial_type subject   wave           beta  \
38660    6.38890   0.91166    correct-go  DEV164  wave1  beta_0005.nii   
38662   11.77224   2.00556  correct-stop  DEV164  wave1  beta_0007.nii   
38664   15.65280   0.67040    correct-go  DEV164  wave1  beta_0009.nii   
38666   20.53475   0.57541    correct-go  DEV164  wave1  beta_0011.nii   
38668   23.35212   0.58260    correct-go  DEV164  wave1  beta_0013.nii   
...          ...       ...           ...     ...    ...            ...   
38900  404.11667   0.65931    correct-go  DEV164  wave1  beta_0245.nii   
38902  407.24723   0.73844    correct-go  DEV164  wave1  beta_0247.nii   
38904  409.68960   2.00556  correct-stop  DEV164  wave1  beta_0249.nii   
38906  412.44516   0.83884    correct-go  DEV164  wa

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:576: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


retrieving a mask from the first image in the series beta_0003.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
0.4021251
76.58324
0.002646322
1.0477667
           onset  duration    trial_type subject   wave           beta  \
39426    2.75834   0.46976    correct-go  DEV168  wave1  beta_0003.nii   
39428    5.51390   0.41272    correct-go  DEV168  wave1  beta_0005.nii   
39430    9.02224   0.41953    correct-go  DEV168  wave1  beta_0007.nii   
39432   12.52780   0.41366    correct-go  DEV168  wave1  beta_0009.nii   
39434   15.90975   0.39443    correct-go  DEV168  wave1  beta_0011.nii   
...          ...       ...           ...     ...    ...            ...   
39668  401.49167   2.00556  correct-stop  DEV168  wave1  beta_0245.nii   
39670  403.99723   0.48803    correct-go  DEV168  wave1  beta_0247.nii   
39672  407.31460   0.46370    correct-go  DEV168  wave1  beta_0249.nii   
39674  410.82016   0.43846    correct-go  DEV168  wave1 

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:576: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_betaseries_nos_30subs_correct_cond_condstd.pkl


In [ ]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs,'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_condstd',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':"beta"
    }
    )

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV006
.................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0009.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-1.2805272
99.762665
0.003390696
1.0808119
         onset  duration    trial_type subject   wave           beta  \
520   15.65280   0.83414    correct-go  DEV006  wave1  beta_0009.nii   
524   23.35212   0.44596    correct-go  DEV006  wave1  beta_0013.nii   
526   25.98407   0.54291    correct-go  DEV006  wave1  beta_0015.nii   
528   28.92644   0.58249    correct-go  DEV006  wave1  beta_0017.nii   
530   31.43061   0.34380    correct-go  DEV006  wave1  beta_0019.nii   
..         ...       ...           ...     ...    ...            ...   
758  407.24723 

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:576: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
1.2027586
66.421265
-0.00096202287
0.964346
          onset  duration    trial_type subject   wave           beta  \
2304    0.00000   2.25834  correct-stop  DEV013  wave1  beta_0001.nii   
2308    5.51390   0.48810    correct-go  DEV013  wave1  beta_0005.nii   
2310    9.02224   0.44515    correct-go  DEV013  wave1  beta_0007.nii   
2312   12.52780   0.53321    correct-go  DEV013  wave1  beta_0009.nii   
2314   15.90975   0.52826    correct-go  DEV013  wave1  beta_0011.nii   
...         ...       ...           ...     ...    ...            ...   
2546  398.86111   0.54400    correct-go  DEV013  wave1  beta_0243.nii   
2550  403.99723   0.45729    correct-go  DEV013  wave1  beta_0247.nii   
2552  407.31460   0.44211    correct-go  DEV013  wave1  beta_0249.nii   
2554  410.82016   0.43882    correct-go  DEV013  wave1  beta_02

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:576: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-0.64897966
109.3175
0.003059946
0.9879997
          onset  duration  trial_type subject   wave           beta  \
2560    0.00000   0.69669  correct-go  DEV014  wave1  beta_0001.nii   
2564    6.38890   0.57531  correct-go  DEV014  wave1  beta_0005.nii   
2568   15.65280   0.65428  correct-go  DEV014  wave1  beta_0009.nii   
2570   20.53475   0.41900  correct-go  DEV014  wave1  beta_0011.nii   
2572   23.35212   0.44371  correct-go  DEV014  wave1  beta_0013.nii   
...         ...       ...         ...     ...    ...            ...   
2802  400.61111   0.38451  correct-go  DEV014  wave1  beta_0243.nii   
2804  404.11667   0.52402  correct-go  DEV014  wave1  beta_0245.nii   
2806  407.24723   0.61086  correct-go  DEV014  wave1  beta_0247.nii   
2810  412.44516   0.59061  correct-go  DEV014  wave1  beta_0251.nii   
2814  421.211

...

retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-0.22936657
89.52612
0.00052254065
1.0213964
           onset  duration    trial_type subject   wave           beta  \
41472    0.00000   0.79588    correct-go  DEV177  wave1  beta_0001.nii   
41476    6.38890   0.84028    correct-go  DEV177  wave1  beta_0005.nii   
41480   15.65280   0.52386    correct-go  DEV177  wave1  beta_0009.nii   
41482   20.53475   0.48580    correct-go  DEV177  wave1  beta_0011.nii   
41484   23.35212   0.39875    correct-go  DEV177  wave1  beta_0013.nii   
...          ...       ...           ...     ...    ...            ...   
41716  404.11667   0.46770    correct-go  DEV177  wave1  beta_0245.nii   
41718  407.24723   0.45717    correct-go  DEV177  wave1  beta_0247.nii   
41720  409.68960   2.00556  correct-stop  DEV177  wave1  beta_0249.nii   
41722  412.44516   0.60240    correct-go  DEV177  wa

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:576: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


retrieving a mask from the first image in the series beta_0003.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
0.81674284
73.743675
0.0011921844
1.0172093
           onset  duration  trial_type subject   wave           beta  \
41730    2.75834   0.93337  correct-go  DEV178  wave1  beta_0003.nii   
41732    5.51390   0.65295  correct-go  DEV178  wave1  beta_0005.nii   
41734    9.02224   0.37882  correct-go  DEV178  wave1  beta_0007.nii   
41736   12.52780   0.50043  correct-go  DEV178  wave1  beta_0009.nii   
41738   15.90975   0.42692  correct-go  DEV178  wave1  beta_0011.nii   
...          ...       ...         ...     ...    ...            ...   
41970  398.86111   0.35703  correct-go  DEV178  wave1  beta_0243.nii   
41974  403.99723   0.41988  correct-go  DEV178  wave1  beta_0247.nii   
41976  407.31460   0.61613  correct-go  DEV178  wave1  beta_0249.nii   
41978  410.82016   0.42749  correct-go  DEV178  wave1  beta_0251.nii   
4

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:576: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
0.87661237
83.04289
